In [10]:
import pandas as pd
import numpy as np
from PIL import Image
import os

def rle2mask(mask_rle, shape):
    '''
    Функция для преобразования RLE маски в 2D numpy массив.
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

# Загрузка метаданных
metadata = pd.read_csv('/kaggle/input/sartorius-cell-instance-segmentation/train.csv')

# Загрузка изображений и масок
data_path = '/kaggle/input/sartorius-cell-instance-segmentation/train'
images = []  # список для хранения изображений
masks = []   # список для хранения масок

for file_name in os.listdir(data_path):
    # Загрузка изображения
    img = Image.open(os.path.join(data_path, file_name))
    img = img.resize((416, 416))  # Изменение размера изображения для совместимости с YOLO
    images.append(np.array(img))
    
    # Загрузка соответствующей маски
    img_id = file_name.split('.')[0]  # Идентификатор изображения
    mask_rle = metadata.loc[metadata['id'] == img_id, 'annotation'].values[0]  # RLE маска
    mask = rle2mask(mask_rle, img.size)
    masks.append(mask)



In [11]:
import torch
# Клонирование репозитория YOLOv5
!git clone https://github.com/ultralytics/yolov5.git

# Переход в папку YOLOv5
os.chdir('yolov5')
!pip install -r requirements.txt  # install
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15705 (delta 9), reused 23 (delta 6), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.44 MiB | 26.04 MiB/s, done.
Resolving deltas: 100% (10755/10755), done.


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-25 Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 134MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [12]:
import cv2

for img in images:
    # Подготовка изображения
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Переход от RGB к BGR, т.к. OpenCV использует BGR
    img = img / 255.0  # Нормализация изображения
    img = torch.from_numpy(img).permute(2, 0, 1).float().to('cuda' if torch.cuda.is_available() else 'cpu')  # Переупорядочивание осей и перевод изображения в тензор PyTorch
    img = img.unsqueeze(0)  # Добавление размерности для пакета

    with torch.no_grad():
        # Прямой проход (forward pass)
        outputs = model(img)

    # Отображение обнаруженных объектов на изображении
    # ...


In [13]:
import os

# Путь к директории
#directory = '/kaggle/working'

# Рекурсивно удалить все файлы в директории и поддиректориях
#for root, dirs, files in os.walk(directory):
#    for file_name in files:
#        file_path = os.path.join(root, file_name)
#os.remove(file_path)

!pip install torchvision yolov5        
import subprocess
packages = [
'aiobotocore<1.29.77,>=1.29.76',
'jupyter-lsp>=2.0.0',
'shapely>=2',
'packaging>=22.0',
'scipy<1.10,>=1.4.1'
]
for package in packages:
    subprocess.call(['pip', 'install', package])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.3/953.3 kB 14.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=90b7c135f4caefa10827d2576594edb82aa87e90a470a622f3e7232718351860
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstal

ERROR: Could not find a version that satisfies the requirement aiobotocore<1.29.77,>=1.29.76 (from versions: 0.0.5, 0.0.6, 0.1.1, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.6.0, 0.6.1a0, 0.7.0, 0.8.0, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.10.0, 0.10.1, 0.10.2, 0.10.3, 0.10.4, 0.11.0, 0.11.1, 0.12.0, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.4.0, 1.4.1, 1.4.2, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.5.0)
ERROR: No matching distribution found for aiobotocore<1.29.77,>=1.29.76


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: jupyter-lsp
    Found existing installation: jupyter-lsp 1.5.1
    Uninstalling jupyter-lsp-1.5.1:
      Successfully uninstalled jupyter-lsp-1.5.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
dask-cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cudf 23.4.1 requires protobuf<4.22,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
cuml 23.4.1 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.
dask-cuda 23.4.0 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.
dask-cudf 23.4.1 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.
distributed 2023.3.2.1 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.1 which is incompatible.
pymc3 3.11.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.1 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.9.3 which is incompatible.


In [14]:
import os
import glob
import json
import shutil
impoty yaml

def process_annotations(json_path, annotations_path):
    with open(json_path) as f:
        data = json.load(f)

    temp_annotations_path = os.path.join(annotations_path, 'temp')
    os.makedirs(temp_annotations_path, exist_ok=True)

    for annotation_id, annotation_data in data['annotations'].items():
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id'] - 1
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox

        image_data = next(image for image in data['images'] if image['id'] == image_id)
        img_width, img_height = image_data['width'], image_data['height']

        x_center = (xmin + width / 2) / img_width
        y_center = (ymin + height / 2) / img_height
        bbox_width = width / img_width
        bbox_height = height / img_height

        annotation_file = os.path.join(temp_annotations_path, f'{annotation_id}.txt')
        with open(annotation_file, 'a') as f:
            f.write(f'{category_id} {x_center} {y_center} {bbox_width} {bbox_height}\n')

    move_annotations(temp_annotations_path, annotations_path)
    os.rmdir(temp_annotations_path)

def move_annotations(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)

    for file_name in os.listdir(source_folder):
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)

        if os.path.exists(destination_path):
            os.remove(destination_path)

        shutil.move(source_path, destination_path)

def create_data_files(image_dir, output_train_file, output_val_file, train_ratio=0.8):
    image_files = glob.glob(os.path.join(image_dir, '*.png'))
    num_images = len(image_files)
    num_train = int(train_ratio * num_images)
    train_files = image_files[:num_train]
    val_files = image_files[num_train:]

    with open(output_train_file, 'w') as f:
        for path in train_files:
            f.write(path + '\n')

    if output_val_file:
        with open(output_val_file, 'w') as f:
            for path in val_files:
                f.write(path + '\n')

# Путь к папке LIVECell_dataset_2021
dataset_folder = '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021'

# Путь к папке, где будут храниться аннотации в формате YOLO
yolo_annotations_path = '/kaggle/temp/data/annotations'

# Обработка аннотаций для папки LIVECell
livecell_annotations_folder = os.path.join(dataset_folder, 'annotations/LIVECell')
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_test.json'), yolo_annotations_path)
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_train.json'), yolo_annotations_path)
process_annotations(os.path.join(livecell_annotations_folder, 'livecell_coco_val.json'), yolo_annotations_path)

# Обработка аннотаций для папки LiveCell_single_cells
livecell_single_cells_annotations_folder = os.path.join(dataset_folder, 'annotations/LIVECell_single_cells')

for category_folder in os.listdir(livecell_single_cells_annotations_folder):
    category_folder_path = os.path.join(livecell_single_cells_annotations_folder, category_folder)
    
    if os.path.isdir(category_folder_path):
        test_file = f'livecell_{category_folder}_test.json'
        train_file = f'livecell_{category_folder}_train.json'
        val_file = f'livecell_{category_folder}_val.json'
        
        process_annotations(os.path.join(category_folder_path, test_file), yolo_annotations_path)
        process_annotations(os.path.join(category_folder_path, train_file), yolo_annotations_path)
        process_annotations(os.path.join(category_folder_path, val_file), yolo_annotations_path)

# Путь к папке с изображениями для train.txt и val.txt
livecell_train_val_images_folder = os.path.join(dataset_folder, 'images/livecell_train_val_images')

# Создание train.txt и val.txt
create_data_files(livecell_train_val_images_folder, '/kaggle/temp/data/annotations/train.txt',
                  '/kaggle/temp/data/annotations/val.txt', train_ratio=0.8)

# Путь к папке с изображениями для test.txt
livecell_test_images_folder = os.path.join(dataset_folder, 'images/livecell_test_images')

# Создание test.txt
create_data_files(livecell_test_images_folder, '/kaggle/temp/data/annotations/test.txt', '')

# Путь к папке с изображениями для test.txt (дополнительная папка test)
test_images_folder = '/kaggle/input/sartorius-cell-instance-segmentation/test'

# Создание test.txt для дополнительной папки test
create_data_files(test_images_folder, '/kaggle/temp/data/annotations/test.txt', '')



In [52]:
import os
import yaml
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import torch
import matplotlib.pyplot as plt

# Путь к папке для сохранения YAML-файла
output_folder = '/kaggle/temp/data'
os.makedirs(output_folder, exist_ok=True)

# The structure of the data for YAML
data = {
    'nc': 2,  # Количество классов
    'names': ['neuron', 'background'], # Имена классов
    'depth_multiple': 0.33,
    'width_multiple':0.50,
    'anchors': [
        [10, 13, 16, 30, 33, 23],
        [30, 61, 62, 45, 59, 119],
        [116, 90, 156, 198, 373, 326]
    ],
    'backbone': [
        [-1, 1, 'Focus', [64, 3]],
        [-1, 1, 'Conv', [128, 3, 2]],
        [-1, 3, 'BottleneckCSP', [128]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [-1, 9, 'BottleneckCSP', [256]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [-1, 9, 'BottleneckCSP', [512]],
        [-1, 1, 'Conv', [1024, 3, 2]],
        [-1, 1, 'SPP', [1024, [5, 9, 13]]],
        [-1, 3, 'BottleneckCSP', [1024, False]]
    ],
    'head': [
        [-1, 1, 'Conv', [512, 1, 1]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 6], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [512, False]],
        [-1, 1, 'Conv', [256, 1, 1]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 4], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [256, False]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [[-1, 14], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [512, False]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [[-1, 10], 1, 'Concat', [1]],
        [-1, 3, 'BottleneckCSP', [1024, False]],
        [[17, 20, 23], 1, 'Detect', [2, [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]]]]
    ],
    'train': '/kaggle/temp/data/annotations/train.txt',  # Путь к файлу train.txt
    'val': '/kaggle/temp/data/annotations/val.txt',  # Путь к файлу val.txt
    'test': '/kaggle/temp/data/annotations/test.txt'  # Путь к файлу test.txt
}



# Путь к файлу YAML
yaml_file_path = os.path.join(output_folder, 'yaml_file.yaml')

# Запись словаря в файл YAML
with open(yaml_file_path, 'w') as f:
    yaml.dump(data, f)

In [16]:
import torch
# Клонирование репозитория YOLOv5
!git clone https://github.com/ultralytics/yolov5.git

# Переход в папку YOLOv5
os.chdir('yolov5')
!pip install -r requirements.txt  # install
model = torch.hub.load("ultralytics/yolov5", "yolov5n")

Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15705 (delta 9), reused 23 (delta 6), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.44 MiB | 25.15 MiB/s, done.
Resolving deltas: 100% (10755/10755), done.


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-25 Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

100%|██████████| 3.87M/3.87M [00:00<00:00, 57.7MB/s]

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [17]:
import os
from yolov5 import train
import cv2
import pandas as pd
import requests

def download_weights(url, save_path):
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as f:
        f.write(response.content)
# URL для загрузки весов модели YOLOv5
url = 'https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5n.pt'
save_path = '/kaggle/working/yourweights.pth'
# Загрузка весов YOLOv5 с GitHub
weights = download_weights(url, save_path)
#weights = 'yolov5s.pt'  # Путь к предварительно обученным весам модели


wandb: W&B disabled due to login timeout.


In [18]:
import torch
import os

!git clone https://github.com/ultralytics/yolov5.git

# Переход в папку YOLOv5
os.chdir('yolov5')
!pip install -r requirements.txt  # install

model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# Путь к файлу yaml с конфигурацией обучения
data_yaml = '/kaggle/temp/data/yaml_file.yaml'

fatal: destination path 'yolov5' already exists and is not an empty directory.


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-25 Python-3.10.10 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [54]:
# Настройки обучения
batch_size = 16
epochs = 10

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# Я предположил, что вы используете yolov5s
from yolov5.models.yolo import Model
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.utils.loss import ComputeLoss

# Путь к файлу yaml с конфигурацией обучения
#data_yaml = '/kaggle/temp/data/yaml_file.yaml'

# Загрузка данных для обучения
dataset = LoadImagesAndLabels(livecell_train_val_images_folder)

# Создание DataLoader для обучения
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# Определение модели
model = Model('/kaggle/temp/data/yaml_file.yaml')

# Определение функции потерь
criterion = ComputeLoss

# Определение оптимизатора
learning_rate = 0.001
optimizer = Adam(model.parameters(), lr=learning_rate)

# Определение шедулера для изменения learning rate
lr_step_size = 5
lr_gamma = 0.1
scheduler = StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# Перемещение модели на GPU (если доступен)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Цикл обучения
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0
    log_interval = 10

    for batch_idx, (images, targets) in enumerate(dataloader):
        images = images.to(device)
        targets = targets.to(device)
        
        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход через модель
        outputs = model(images)
        
        # Расчет функции потерь
        loss, loss_items = criterion(outputs, targets)

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()
        
        # Суммарная потеря во время эпохи
        total_loss += loss.item()
        total_samples += targets.shape[0]
        
        # Вывод информации о процессе обучения
        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item():.4f}")
    
    # Печать средней потери в конце эпохи
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
    
    # Обновление learning rate
    scheduler.step()

# Сохранение обученной модели
# torch.save(model.state_dict(), saved_model_path)



Scanning /kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/labels/livecell_train_val_images/A172... 0 images, 4184 backgrounds, 0 corrupt: 100%|██████████| 4184/4184 [00:06<00:00, 672.89it/s]
WARNING ⚠️ No labels found in /kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/labels/livecell_train_val_images/A172.cache. See https://docs.ultralytics.com/yolov5/tutorials/train_custom_data
WARNING ⚠️ Cache directory /kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/labels/livecell_train_val_images is not writeable: [Errno 2] No such file or directory: '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/labels/livecell_train_val_images/A172.cache.npy'

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.

ValueError: too many values to unpack (expected 2)

In [48]:
import yaml

def check_yaml_file(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)

    for i, item in enumerate(data['backbone'] + data['head']):
        if len(item) != 4:
            print(f"Item at index {i} does not contain exactly four elements. It contains {len(item)} elements: {item}")

# замените 'your_yaml_file_path.yaml' на путь к вашему YAML файлу
check_yaml_file('/kaggle/temp/data/yaml_file.yaml')

Item at index 0 does not contain exactly four elements. It contains 10 elements: [[-1, 1, 'Focus', [64, 3]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'BottleneckCSP', [128]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 9, 'BottleneckCSP', [256]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 9, 'BottleneckCSP', [512]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 1, 'SPP', [1024, [5, 9, 13]]], [-1, 3, 'BottleneckCSP', [1024, False]]]
Item at index 1 does not contain exactly four elements. It contains 15 elements: [[-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'BottleneckCSP', [512, False]], [-1, 1, 'Conv', [256, 1, 1]], [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 3, 'BottleneckCSP', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 14], 1, 'Concat', [1]], [-1, 3, 'BottleneckCSP', [512, False]], [-1, 1, 'Conv', [512, 3, 2]], [[-1, 10], 1, 'Concat', [1]], [-1, 3, 'BottleneckCSP', [1024, False]], [[17, 20, 23], 1, 'De

In [25]:
with open('/kaggle/temp/data/yaml_file.yaml', 'r') as file:
    print(file.read())

classes:
- neuron
- background
test: /kaggle/temp/data/annotations/test.txt
train: /kaggle/temp/data/annotations/train.txt
val: /kaggle/temp/data/annotations/val.txt



In [21]:
import os

# Путь к директории с изображениями
image_dir = '/kaggle/input/sartorius-cell-instance-segmentation/train'

# Проверка существования директории
if not os.path.isdir(image_dir):
    print('Указанная директория не существует')
else:
    # Получение списка файлов с расширением изображений
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Проверка наличия изображений
    if len(image_files) == 0:
        print('В указанной директории нет изображений')
    else:
        print('Изображения найдены')
        print('Количество изображений:', len(image_files))

Изображения найдены
Количество изображений: 606


In [ ]:

# Запуск обучения
#!python train.py --data /kaggle/temp/data/yaml_file.yaml --epochs 10 --weights '/kaggle/working/yourweights.pth' --cfg yolov5s.yaml  --batch-size 16

In [ ]:
#!pip install clearml
#!pip install comet_ml